## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-21-50-04 +0000,startribune,Trump slams his own supporters as 'weaklings' ...,https://www.startribune.com/trump-slams-his-ow...
1,2025-07-16-21-48-16 +0000,bbc,US says 'specific steps' agreed to end Syria v...,https://www.bbc.com/news/articles/cp90l77187zo
2,2025-07-16-21-40-27 +0000,bbc,Two dead and many injured in Russian strike on...,https://www.bbc.com/news/articles/c62d17ld032o
3,2025-07-16-21-38-43 +0000,nyt,Eswatini Says It Will Repatriate Migrants Depo...,https://www.nytimes.com/2025/07/16/world/afric...
4,2025-07-16-21-38-07 +0000,bbc,What we know so far about Afghan data breach,https://www.bbc.com/news/articles/c79qyl907lxo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2305/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,12
206,pm,4
17,strikes,4
29,will,4
55,israel,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2025-07-16-21-50-04 +0000,startribune,Trump slams his own supporters as 'weaklings' ...,https://www.startribune.com/trump-slams-his-ow...,27
15,2025-07-16-19-56-53 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...,26
3,2025-07-16-21-38-43 +0000,nyt,Eswatini Says It Will Repatriate Migrants Depo...,https://www.nytimes.com/2025/07/16/world/afric...,25
9,2025-07-16-20-53-24 +0000,bbc,Trump discussed firing Fed boss but 'highly un...,https://www.bbc.com/news/articles/c4geyrdprwjo,25
18,2025-07-16-19-15-59 +0000,bbc,Canada curbs steel imports to shield domestic ...,https://www.bbc.com/news/articles/cy5wv7rzvk0o,25


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,27,2025-07-16-21-50-04 +0000,startribune,Trump slams his own supporters as 'weaklings' ...,https://www.startribune.com/trump-slams-his-ow...
1,19,2025-07-16-21-48-16 +0000,bbc,US says 'specific steps' agreed to end Syria v...,https://www.bbc.com/news/articles/cp90l77187zo
59,17,2025-07-16-03-00-00 +0000,world,"Through Trial and Error, Iran Found Gaps in Is...",https://www.wsj.com/world/middle-east/iran-isr...
51,16,2025-07-16-10-35-23 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
15,14,2025-07-16-19-56-53 +0000,nyt,Carney Moves to Reduce Canada’s Chinese Steel ...,https://www.nytimes.com/2025/07/16/world/canad...
29,14,2025-07-16-17-12-33 +0000,startribune,A $70 kosher pastrami sandwich? That’s what on...,https://www.startribune.com/minneapolis-dfl-co...
28,14,2025-07-16-17-22-24 +0000,bbc,Serious questions for Tories over Afghan data ...,https://www.bbc.com/news/articles/c98w2e9leywo
19,13,2025-07-16-19-10-07 +0000,startribune,Riders have been slow to hop on the new Gold L...,https://www.startribune.com/riders-have-been-s...
49,13,2025-07-16-12-20-00 +0000,world,Nvidia CEO Lavishes Praise on China in Beijing...,https://www.wsj.com/world/asia/nvidia-ceo-lavi...
24,12,2025-07-16-18-23-44 +0000,startribune,Vance Boelter plans to plead not guilty in sho...,https://www.startribune.com/vance-boelter-plan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
